# Cards Issued Difference  
**JPMorgan SQL Interview Question**

## Question  
Your team at JPMorgan Chase is preparing to launch a new credit card, and to gain some insights, you're analyzing how many credit cards were issued each month.

Write a query that outputs the name of each credit card and the difference in the number of issued cards between the month with the highest issuance cards and the lowest issuance. Arrange the results based on the largest disparity.

---

## monthly_cards_issued Table:

| Column Name        | Type     |
|--------------------|----------|
| card_name          | string   |
| issued_amount      | integer  |
| issue_month        | integer  |
| issue_year         | integer  |

---

## Example Input:

| card_name              | issued_amount | issue_month | issue_year |
|------------------------|---------------|-------------|------------|
| Chase Freedom Flex     | 55000         | 1           | 2021       |
| Chase Freedom Flex     | 60000         | 2           | 2021       |
| Chase Freedom Flex     | 65000         | 3           | 2021       |
| Chase Freedom Flex     | 70000         | 4           | 2021       |
| Chase Sapphire Reserve | 170000        | 1           | 2021       |
| Chase Sapphire Reserve | 175000        | 2           | 2021       |
| Chase Sapphire Reserve | 180000        | 3           | 2021       |

---

## Example Output:

| card_name              | difference |
|------------------------|------------|
| Chase Freedom Flex     | 15000      |
| Chase Sapphire Reserve | 10000      |

---

## Explanation:  
- *Chase Freedom Flex's* best month was 70,000 cards issued, and the worst month was 55,000 cards, so the difference is 15,000 cards.  
- *Chase Sapphire Reserve’s* best month was 180,000 cards issued and the worst month was 170,000 cards, so the difference is 10,000 cards.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import *

# Create a Spark session
spark = SparkSession.builder.master('local[1]').appName("CardsIssuedDifference").getOrCreate()

# Define schema
schema = StructType([
    StructField("card_name", StringType(), True),
    StructField("issued_amount", IntegerType(), True),
    StructField("issue_month", IntegerType(), True),
    StructField("issue_year", IntegerType(), True)
])

# Sample data
data = [
    ("Chase Freedom Flex", 55000, 1, 2021),
    ("Chase Freedom Flex", 60000, 2, 2021),
    ("Chase Freedom Flex", 65000, 3, 2021),
    ("Chase Freedom Flex", 70000, 4, 2021),
    ("Chase Sapphire Reserve", 170000, 1, 2021),
    ("Chase Sapphire Reserve", 175000, 2, 2021),
    ("Chase Sapphire Reserve", 180000, 3, 2021),
]

# Create DataFrame
df = spark.createDataFrame(data, schema)

# Show the DataFrame
df.show()


+--------------------+-------------+-----------+----------+
|           card_name|issued_amount|issue_month|issue_year|
+--------------------+-------------+-----------+----------+
|  Chase Freedom Flex|        55000|          1|      2021|
|  Chase Freedom Flex|        60000|          2|      2021|
|  Chase Freedom Flex|        65000|          3|      2021|
|  Chase Freedom Flex|        70000|          4|      2021|
|Chase Sapphire Re...|       170000|          1|      2021|
|Chase Sapphire Re...|       175000|          2|      2021|
|Chase Sapphire Re...|       180000|          3|      2021|
+--------------------+-------------+-----------+----------+



In [2]:
df.groupBy('card_name').agg((max('issued_amount')-min('issued_amount')).alias('difference'))\
    .orderBy('difference').show()

+--------------------+----------+
|           card_name|difference|
+--------------------+----------+
|Chase Sapphire Re...|     10000|
|  Chase Freedom Flex|     15000|
+--------------------+----------+



In [3]:
df.createOrReplaceTempView('monthly_cards_issued')

spark.sql(''' 
SELECT card_name, max(issued_amount)-min(issued_amount) as difference
FROM monthly_cards_issued
group by card_name
order by 2 desc
''').show()

+--------------------+----------+
|           card_name|difference|
+--------------------+----------+
|  Chase Freedom Flex|     15000|
|Chase Sapphire Re...|     10000|
+--------------------+----------+

